# Validación de modelos

A la hora de crear nuestros modelos es importante que podamos validar los resultados que nos están proporcionando y así tener una idea clara de la fiabilidad de dichos modelos. 

En este notebook vamos a explorar algunas de las técnicas de validación de modelos que podemos utilizar. 

## Validación simple

Es el método más sencillo de validación que consiste en repartir aleatoriamente las observaciones en dos grupos, el set de entrenamiento y el set de pruebas. 

Es una forma de validación fácil y rápida de implementar gracias a la funcionalidad de train_test_split que nos ofrece la libreria de scikit-learn pero tenemos que ser conscientes de que tiene dos problemas: 
- La estimación del error es variable dependiendo de qué observaciones se incluyan en el set de entrenamiento y el set de pruebas. Podemos minimizar este impacto fijando la semilla de los números pseudoaleatorios que utilizamos para la separación de los sets de entrenamiento y pruebas. 
- Al excluir parte de los datos del entrenamiento del modelo y dejarlos reservados para las pruebas, disminuimos la información disponible para el modelo y por lo tanto su capacidad predictiva. Podemos minimizar este impacto asegurándonos de tener un set de datos amplio y que refleje los distintos comportamientos de los datos y así no mermar la capacidad predictiva del modelo. 



Vamos a ver un ejemplo de como empleamos el método de validación simple. 

In [1]:
from sklearn.datasets import make_blobs
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
# Vamos a crear un set de datos que utilizaremos en todos los ejemplos para comparar su eficacia
X, y = make_blobs(n_samples=500,random_state =123)

In [3]:
# vamos a crear nuestro set de entrenamiento y de test 
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123)

In [4]:
# Creamos el modelo que vamos a utilizar con todos los métodos de validación para poder compararlo
# En este caso será un clasificador utilizando un Random Forest

rf = RandomForestClassifier()

In [5]:
#Entrenamos el modelo con los datos del set de entrenamiento
rf.fit(X_train, y_train)

RandomForestClassifier()

In [6]:
# Generamos predicciones para el set de pruebas
y_pred = rf.predict(X_test)


In [9]:
# Finalmente calculamos la precisión para el modelo
precision_rf = accuracy_score(y_test, y_pred)
print("Precision validación simple: %.3f" % precision_rf)

Precision validación simple: 0.944


## Leave One Out Cross-Validation (LOOCV)

El método LOOCV es un método iterativo que se inicia empleando como set de entrenamiento todas las observaciones disponibles en nuestro set de datos menos una, que se excluye para emplearla como validación. 

Si se emplea una única observación para calcular el error del modelo, este varía mucho dependiendo de qué observación se haya seleccionado. Para evitarlo, el proceso se repite tantas veces como observaciones disponibles, excluyendo en cada iteración una observación distinta, entrenando el modelo con el resto y calculando el error con dicha observación. Finalmente, el error estimado por el método LOOCV es el promedio de todos los errores calculados durante la iteración del proceso. 

Este método LOOCV permite reducir la variabilidad que se origina si se divide aleatoriamente las observaciones en dos grupos. Esto es así porque al final del proceso de LOOCV se acaban empleando todos los datos disponibles tanto para entrenamiento como para validación. Al no haber una separación aleatoriade los datos, los resultados de LOOCV son reproducibles. 

La principal desventaja de este método es su coste computacional. El proceso requiere que el modelo sea reajustado y validado tantas veces como observaciones disponibles, lo que en algunos casos puede ser muy complicado. 

LOOCV es un proceso de validación muy extendido ya que puede aplicarse para evaluar cualquier tipo de modelo. Sin embargo, hay distintos autores de publicaciones sobre estadística que consideran que, al emplearse todas las observaciones como entrenamiento, se puede estar cayendo en el overfitting y, aunque lo consideran un método aceptable, aconsejan utilizar otros métodos como K-Fold Cross-Validation (KFCV)



Ahora vamos a ver cómo hacemos la validación LOOCV en python con el mismo ejemplo que hemos utilizado en el método de validación simple:

In [10]:
from sklearn.model_selection import LeaveOneOut

In [11]:
# Creamos el objeto que nos permite hacer la separación de los datos
cv = LeaveOneOut()

In [12]:
# Necesitamos guardar tanto los valores reales como las predicciones en cada iteración 
# así que creamos dos listas para esas tareas

y_true_loocv, y_pred_loocv = list(), list()

In [13]:
# Empezamos el proceso iterativo de validación del modelo utilizando el mismo tipo de modelo que en el 
# ejemplo de validación simple

for train_ix, test_ix in cv.split(X):
  # hacemos la separación de los sets de entrenamiento y pruebas
  X_train, X_test = X[train_ix, :], X[test_ix, :]
  y_train, y_test = y[train_ix], y[test_ix]
  # Entrenamos el modelo
  modelo = RandomForestClassifier()
  modelo.fit(X_train, y_train)
  # generamos predicciones
  y_hat = modelo.predict(X_test)
  # guardamos el valor real y la predicción para su evaluación más adelante
  y_true_loocv.append(y_test[0])
  y_pred_loocv.append(y_hat[0])

In [14]:
# finalmente calculamos la precisión del modelo con este tipo de validación
precision_loocv = accuracy_score(y_true_loocv, y_pred_loocv)

print("Precision validacion LOOCV: %.3f" % precision_loocv)

Precision validacion LOOCV: 0.950


## K-Fold Cross Validation (KFCV)

El método K-Fold Cross Validation (KFCV) es al igual que LOOCV un método iterativo de validación de modelos. 

Consiste en dividir los datos de forma aleatoria en *k* grupos de aproximadamente el mismo tamaño, donde *k-1* grupos se emplean para entrenar el modelo y uno de los grupos se emplea como validación. Este proceso se repite durante *k* veces utilizando un grupo distinto como validación en cada iteración. El proceso genera *k* estimaciones del error y el error final se calcula como la media de las distintas medidas de error durante el proceso iterativo. 

KFCV presenta dos ventajas frente a LOOCV: 

- Es menos intenso computacionalmente, dado que el número de iteraciones necesarias viene determinado por el valor *k* escogido. Por lo general, se recomienda utilizar un valor de *k* entre 5 y 10. 

- Gestiona mejor el balance entre bias y varianza. La principal ventaja de KFCV es que consigue una estimación precisa del error de test gracias a un mejor balance entre bias y varianza. LOOCV emplea *n-1* observaciones para entrenar el modelo, lo que es prácticamente todo el set de datos disponible, maximizando así el ajuste del modelo a los datos disponibles y reduciendo el bias. Sin embargo, para la estimación final del error se promedian las estimaciones de *n* modelos entrenados prácticamente con los mismos datos (solo hay una observación de diferencia entre cada set de entrenamiento), por lo que están altamente correlados. Eso se traduce en un mayor riesgo de overfitting. En el método KFCV los *k* grupos empleados como set de entrenamiento se solapan mucho menos, lo que se traduce en un riesgo mucho menor de caer en el overfitting a pesar de utilizar todos los datos disponibles para entrenar el modelo. 






Vamos a ver un ejemplo de aplicación de KFCV sobre el mismo set de datos que hemos utilizado antes

In [19]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from numpy import mean

In [16]:
# Preparamos el objeto que nos va a permitir separar los datos en los distintos grupos

cv = KFold( n_splits=10, random_state = 123, shuffle=True)

In [17]:
# Creamos el modelo que vamos a utilizar, seguimos con el clasificador de RandomForest
rf_kfcv = RandomForestClassifier()

In [18]:
# El método de cross_val_score nos permite entrenar y validar el modelo directamente
medidas = cross_val_score(rf_kfcv, X, y, scoring='accuracy', cv=cv)

In [20]:
precision_kfcv = mean(medidas)

print("Precision KFCV: %.3f" % precision_kfcv)

Precision KFCV: 0.942


## Comparación de métodos de validación

En general no existe un método de validación que supere al resto en todos los escenarios, y la elección debe basarse en diversos factores: 

- Si el tamaño de la muestra es pequeño, se recomienda utilizar KFCV ya que consigue un buen equilibrio entre bias y varianza. 

- Si el tamaño de la muestra es muy grande, la diferencia entre los distintos métodos se reduce mucho y se debe valorar la eficiencia computacional. KFCV con k=5 o una validación simple son suficientes para obtener buenas estimaciones de las métricas del modelo. 
